In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search
)

In [4]:
output_dir = "talklife_moc_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

Talklife MoC

In [5]:
%run load_talklifemoc.py

In [6]:
%run load_sbert-embeddings.py

In [7]:
sbert_embeddings.shape

torch.Size([18604, 384])

swmhau Network

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
num_features = len(features)
add_time_in_path = False

In [9]:
num_epochs = 100
embedding_dim = 384
dimensions = [15] # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[64,64], [128,128]]#[[256,256],[512,512]]
dropout_rates = [0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
#create indices for kfold
fold_col_names = [c for c in df.columns if 'fold' in c ]
fold_list = []
for foldc in fold_col_names:
    fold_list.append((df[df[foldc]=='train'].index, df[df[foldc]=='dev'].index, df[df[foldc]=='test'].index))
fold_list = tuple(fold_list)

In [11]:
size = 5
swmhau_network_umap, best_swmhau_network_umap, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_indices=fold_list,
    k_fold=True,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=add_time_in_path,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/18604 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/18604 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in talklife_moc_output/swmhau_network_umap_focal_2_5.csv
saving the best model results dataframe to CSV for this hyperparameter search in talklife_moc_output/swmhau_network_umap_focal_2_5_best_model.csv


In [19]:
best_swmhau_network_umap

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.809073,0.547445,"[0.8888384901992548, 0.48662019338880147, 0.26...",0.533280,"[0.9047556229800146, 0.4450843274372686, 0.25]",0.565459,"[0.8734717269485481, 0.5367063492063492, 0.286...",None,0.802009,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.808160,0.563439,"[0.8873244027248134, 0.4848353552859619, 0.318...",0.545084,"[0.9088602523869934, 0.43038461538461537, 0.29...",0.588579,"[0.8667855323484462, 0.5550595238095238, 0.343...",None,0.799466,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.816921,0.559617,"[0.8941957523606948, 0.4789437741862053, 0.305...",0.550163,"[0.905252854812398, 0.44257467395877154, 0.302...",0.571351,"[0.8834055017829853, 0.5218253968253969, 0.308...",None,0.809262,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [20]:
best_swmhau_network_umap["f1"].mean()

0.556833858216276

In [21]:
best_swmhau_network_umap["precision"].mean()

0.5428423211687781

In [22]:
best_swmhau_network_umap["recall"].mean()

0.5751297842300634

In [23]:
np.stack(best_swmhau_network_umap["f1_scores"]).mean(axis=0)

array([0.89011955, 0.48346644, 0.29691559])

In [24]:
np.stack(best_swmhau_network_umap["precision_scores"]).mean(axis=0)

array([0.90628958, 0.43934787, 0.28288951])

In [25]:
np.stack(best_swmhau_network_umap["recall_scores"]).mean(axis=0)

array([0.87455425, 0.53786376, 0.31297134])